In [1]:
import pandas as pd
import numpy as np

In [2]:
train= pd.read_csv('train.csv')
test= pd.read_csv('test.csv')
policies= pd.read_csv('policies.csv')

In [3]:
def fill_pol(df):
    idk=policies[['policy_number', 'product_code','agent', 'class_of_business', 'client_type', 'renewal_frequency','branch']]
    idk.rename({'product_code':'product'}, axis=1, inplace=True)
    df[df['policy_number'].isnull()==True]
    fill_pol=df[df['policy_number'].isnull()==True].drop(['policy_number'], axis=1).merge(idk, on=['product', 'agent', 'class_of_business', 'client_type', 'renewal_frequency','branch'], how='inner')
    null_=fill_pol[['claim_id', 'policy_number']].groupby(by='claim_id', as_index=False, axis=0).agg(lambda x: pd.Series.mode(x)[0])
    filled=df[df['policy_number'].isnull()==True].drop(['policy_number'], axis=1).merge(null_, on=['claim_id'], how='inner')
    see= pd.concat([df[df['policy_number'].isnull()==False], filled], axis=0)
    return see

In [4]:
target= train['target']
train_test= pd.concat([train.drop(['target'], axis=1), test], axis=0).drop(['claim_number', 'claim_id', 'policy_number'], axis=1)

In [5]:
from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(train_test)
categorical_columns = categorical_columns_selector(train_test)



In [6]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

numerical_preprocessor = RobustScaler()


categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value",
                                          unknown_value=-1)

preprocessor = ColumnTransformer([
    ('ordinal-encoder', categorical_preprocessor, categorical_columns),
    ('standard-scaler', numerical_preprocessor, numerical_columns)])



C:\Users\ASUS\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:17: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  "Since version 1.0, "


In [7]:
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression, HuberRegressor, QuantileRegressor, RANSACRegressor, TheilSenRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_validate, validation_curve
from sklearn.model_selection import cross_validate, ShuffleSplit
from sklearn.preprocessing import PowerTransformer, RobustScaler, MaxAbsScaler, Normalizer
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import BaggingRegressor,RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, VotingRegressor

In [8]:
from sklearn.preprocessing import PolynomialFeatures
m= target.shape[0]

In [9]:
train_test_np= pd.DataFrame(preprocessor.fit_transform(train_test), columns=train_test.columns)

In [10]:
train_test_np

,sum_insured,product,agent,class_of_business,risk_type,loss_date,client_type,renewal_frequency,primary_cause,secondary_cause,branch
0,26.0,4.0,30.0,5.0,1220.0,2.0,0.0,6.0,20.0,5.0,0.568129
1,4.0,7.0,9.0,31.0,132.0,2.0,0.0,6.0,21.0,1.0,-0.087760
2,28.0,1.0,9.0,25.0,229.0,2.0,0.0,6.0,22.0,1.0,-0.117783
3,4.0,1.0,8.0,31.0,353.0,2.0,0.0,6.0,19.0,0.0,-0.110855
4,14.0,7.0,32.0,22.0,310.0,0.0,0.0,6.0,19.0,0.0,-0.062702
...,...,...,...,...,...,...,...,...,...,...,...
13390,14.0,1.0,32.0,22.0,1157.0,0.0,4.0,6.0,19.0,1.0,4.494226
13391,4.0,4.0,8.0,31.0,220.0,2.0,1.0,6.0,21.0,6.0,-0.108545
13392,14.0,7.0,32.0,22.0,1141.0,0.0,1.0,6.0,19.0,1.0,22.969977
13393,14.0,3.0,32.0,22.0,1074.0,0.0,1.0,6.0,2.0,4.0,14.309469


In [11]:
import matplotlib.pyplot as plt

In [12]:
polyfeat=PolynomialFeatures(degree=3,interaction_only=True, include_bias=False).fit_transform(train_test_np)
cols=[str(i) for i in range(polyfeat.shape[1])]
polydf=pd.DataFrame(polyfeat, columns=cols)
tog=pd.concat([polydf.iloc[:m,:], train['target']], axis=1)
cor=list(tog.corr()[tog.corr()['target'].abs() >=0.1].index)
print(tog.corr()[tog.corr()['target'].abs() >=0.1]['target'])
cor.remove('target')
feat_train_test= pd.concat([train_test_np.reset_index(drop=True), polydf[cor].reset_index(drop=True)], axis=1, ignore_index=True)
feat_train_test

4         0.260388
10        0.102604
14        0.176657
23        0.134753
29        0.124426
31        0.184762
37        0.100664
38        0.168736
39       -0.104809
46        0.166554
47        0.180282
48        0.185639
50        0.103601
59        0.177214
64        0.132165
76        0.152517
83        0.157554
91        0.113191
92        0.145504
93        0.155060
104       0.102534
112       0.111929
118       0.110074
119       0.101215
127       0.108195
128       0.108148
129       0.113502
131       0.124885
140       0.198574
145       0.108169
147       0.165887
155       0.108286
156       0.168126
157       0.162097
168       0.118505
176       0.112655
177       0.120505
178       0.127023
201       0.115858
202       0.128906
204       0.170804
205       0.141469
209       0.136224
225       0.188468
target    1.000000
Name: target, dtype: float64


,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,26.0,4.0,30.0,5.0,1220.0,2.0,0.0,6.0,20.0,5.0,...,0.000000,0.0,36600.0,122000.0,0.0,0.0,0.000000,146400.0,13862.355658,0.000000
1,4.0,7.0,9.0,31.0,132.0,2.0,0.0,6.0,21.0,1.0,...,-0.000000,0.0,24552.0,85932.0,0.0,0.0,-0.000000,16632.0,-243.270208,-0.000000
2,28.0,1.0,9.0,25.0,229.0,2.0,0.0,6.0,22.0,1.0,...,-0.000000,0.0,34350.0,125950.0,0.0,0.0,-0.000000,30228.0,-593.390300,-0.000000
3,4.0,1.0,8.0,31.0,353.0,2.0,0.0,6.0,19.0,0.0,...,-0.000000,0.0,65658.0,207917.0,0.0,0.0,-0.000000,40242.0,-743.501155,-0.000000
4,14.0,7.0,32.0,22.0,310.0,0.0,0.0,6.0,19.0,0.0,...,-0.000000,0.0,40920.0,129580.0,0.0,0.0,-0.000000,35340.0,-369.315242,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13390,14.0,1.0,32.0,22.0,1157.0,0.0,4.0,6.0,19.0,1.0,...,575.260970,101816.0,152724.0,483626.0,27768.0,87932.0,20799.279446,131898.0,98796.577367,341.561201
13391,4.0,4.0,8.0,31.0,220.0,2.0,1.0,6.0,21.0,6.0,...,-0.868360,6820.0,40920.0,143220.0,1320.0,4620.0,-23.879908,27720.0,-501.478060,-2.279446
13392,14.0,7.0,32.0,22.0,1141.0,0.0,1.0,6.0,19.0,1.0,...,735.039261,25102.0,150612.0,476938.0,6846.0,21679.0,26208.743649,130074.0,497966.129330,436.429561
13393,14.0,3.0,32.0,22.0,1074.0,0.0,1.0,6.0,2.0,4.0,...,457.903002,23628.0,141768.0,47256.0,6444.0,2148.0,15368.369515,12888.0,30736.739030,28.618938


In [13]:
def crs_val(algo):
  # run block of code and catch warnings
  import warnings
  with warnings.catch_warnings():
    #warnings.filterwarnings("ignore")
    cv= ShuffleSplit(n_splits=10, random_state=15, test_size=0.36)
    model= Pipeline(steps=[ ('model', algo)])
    result= cross_validate(model, feat_train_test.iloc[:target.shape[0], :], target, cv=cv, scoring='neg_mean_absolute_error', return_train_score=False)
    #print(result)
    return -result['test_score']

In [17]:
algos= {'HuberRegressor':HuberRegressor(max_iter=2000)
       }
for alg in algos.keys():
    score= crs_val(algos[alg])
    print(alg+': \n',score.mean(), ' +/- ',score.std())

HuberRegressor: 
 39503.15092326341  +/-  3164.442101497294


HuberRegressor: 
 40425.73638794304  +/-  2898.65071809174
LinearRegression: 
 58812.296777605676  +/-  1459.4150668320804
AdaBoostRegressor: 
 173693.8801544674  +/-  66538.28400556999
ExtraTreesRegressor: 
 47472.53445066674  +/-  2146.962893629694
GradientBoostingRegressor: 
 45738.132439766836  +/-  2550.0362847896185
XGB: 
 48187.95602647209  +/-  2862.478462897075
LGB: 
 47658.83402736619  +/-  1378.2097632031728

In [18]:
algo= Pipeline(steps=[('model', HuberRegressor(max_iter=2000))])
algo.fit(feat_train_test.iloc[:m,:], target)
pred= algo.predict(feat_train_test.iloc[m:,:])
pred

array([ 7732.32741418, 15829.57169209,  9944.19856075, ...,
       24480.1531581 , 20309.83762026, 16658.78207603])

In [19]:
def submit():
    import datetime
    sub= pd.read_csv('SampleSubmission.csv')
    sub['target']=pred
    moment=datetime.datetime.now().strftime('%m%d%H%M%S')
    #print(moment)
    sub.to_csv(moment+'.csv', index=False)
    print(moment)
submit()

0319140502


In [20]:
test.columns

Index(['claim_id', 'policy_number', 'claim_number', 'sum_insured', 'product',
       'agent', 'class_of_business', 'risk_type', 'loss_date', 'client_type',
       'renewal_frequency', 'primary_cause', 'secondary_cause', 'branch'],
      dtype='object')

,claim_id,policy_number,claim_number,sum_insured,product,agent,class_of_business,risk_type,loss_date,client_type,renewal_frequency,primary_cause,secondary_cause,branch
0,ID_JZ7QWRHH57RO,pol000009680,claim0011359,300000.0,prod00027,ag00068,cob00031,rt00006,2021-05-13,ct0003,rf0001,pc0007,sc00022,br00006
1,ID_KQ5TJMVLJ4EP,pol000006869,claim0005272,16000.0,prod00005,ag00226,cob00010,rt00032,2018-05-13,ct0003,rf0001,pc0007,sc00023,br00002
2,ID_NYHI7WJGGIAE,pol000006135,claim0004147,3000.0,prod00029,ag00037,cob00010,rt00026,2018-08-18,ct0003,rf0001,pc0007,sc00024,br00002
3,ID_ROMMDCXYUXN5,pol000008991,claim0009962,6000.0,prod00005,ag00037,cob00009,rt00032,2018-12-20,ct0003,rf0001,pc0007,sc00021,br00001
4,ID_2OTD9NX8L73D,pol000007768,claim0007346,26850.0,prod00015,ag00226,cob00034,rt00023,2018-11-07,ct0001,rf0001,pc0007,sc00021,br00001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4014,ID_F08UFUIVVMR8,pol000006124,claim0004124,2000000.0,prod00015,ag00037,cob00034,rt00023,2021-03-11,ct0001,rf0005,pc0007,sc00021,br00002
4015,ID_KODD3CK5SV0X,pol000010377,claim0014092,7000.0,prod00005,ag00068,cob00009,rt00032,2018-08-09,ct0003,rf0002,pc0007,sc00023,br00007
4016,ID_59003DPSZQ16,pol000006080,claim0003914,10000000.0,prod00015,ag00226,cob00034,rt00023,2021-02-23,ct0001,rf0002,pc0007,sc00021,br00002
4017,ID_B0C9XF5SIHGO,pol000009493,claim0010854,6250000.0,prod00015,ag00064,cob00034,rt00023,2020-12-17,ct0001,rf0002,pc0007,sc00004,br00005


In [11]:
pd.read_csv('DataDictionary.csv')

,Policy Data,Unnamed: 1
0,Columns,Data type
1,policy number,Unique policy identifier
2,product code,product code
3,trans date,date of policy transaction
4,from date,first date of policy cover
5,to date,last date of policy cover
6,agency full name,agency
7,class of business,class of business
8,client type,client type
9,Renewal frequency,policy renewal frequency
